<a href="https://colab.research.google.com/github/yizhengf/kaggle_team_week1/blob/master/kaggle_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q kaggle #安装kaggle命令行工具
!mkdir -p ~/.kaggle # 用用户名和apikey 登陆
!echo '{"username":"yizhengf","key":"719be4825d9b94e59ca01cf12d41434e"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# 下载数据
!mkdir -p raw 
!kaggle competitions download -c santander-customer-transaction-prediction -p raw

 92% 113M/122M [00:00<00:00, 97.4MB/s] 
100% 122M/122M [00:01<00:00, 112MB/s] 
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 98.1MB/s]
 99% 121M/122M [00:00<00:00, 124MB/s]
100% 122M/122M [00:00<00:00, 137MB/s]


In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
df=pd.read_csv("raw/train.csv.zip")

In [0]:
# Split data into feature and label: X is feature, y is target,exclude IDs
X = df.iloc[:, 2:]
y = df.iloc[:, 1]

In [6]:
!pip install keras_tqdm #进度条

In [7]:
import keras_tqdm

Using TensorFlow backend.


In [0]:
import tensorflow as tf
import keras

In [0]:

# converting data into numpy arrays for keras
X_array = X.values
y_array = y.values

In [12]:
# check shape of the resulting array
print(X.shape)
print(y.shape)

(200000, 200)
(200000,)


In [16]:
# onehot encoding 因为keras需要onehot，encoding
y_onehot = keras.utils.to_categorical(y, 2)
y_onehot.shape

(200000, 2)

In [0]:
#分两次，把数据分成train/test/validation， 比例0.6/0.2/0。2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_onehot, y_test_onehot = train_test_split(X, 
                                                                  y_onehot,
                                                                  test_size = 0.2,
                                                                  random_state = 42
                                                                 ) 

X_train, X_validation, y_train_onehot, y_validation_onehot = train_test_split(X_train, 
                                                                  y_train_onehot,
                                                                  test_size = 0.25,
                                                                  random_state = 42
                                                                 )

In [18]:
# 验证dimension
print(X_train.shape, X_test.shape, X_validation.shape)
print(y_train_onehot.shape, y_test_onehot.shape, y_validation_onehot.shape)


(120000, 200) (40000, 200) (40000, 200)
(120000, 2) (40000, 2) (40000, 2)


In [0]:
# 开始训练模型
from keras import backend
from keras.layers import Dense, Activation
from keras.models import Sequential

backend.clear_session()

In [20]:
# 定义模型形状和activation function
model = Sequential()  # it is a feed-forward network without loops like in RNN
model.add(Dense(256, input_shape=(200,)))  # the first layer must specify the input shape (replacing placeholders)
model.add(Activation('sigmoid'))
model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               51456     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 117,762
Trainable params: 117,762
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam',
    metrics=['accuracy']
)

In [23]:
from keras_tqdm import TQDMCallback # 进度条

model.fit(
    X_train, 
    y_train_onehot,
    batch_size=512, 
    epochs=40,
    validation_data=(X_validation, y_validation_onehot),
    callbacks=[TQDMCallback()],
    verbose=0
)

Instructions for updating:
Use tf.cast instead.


Epoch: 0 - loss: 0.278, acc: 0.903:  98%|█████████▊| 117760/120000 [00:04<00:00, 31300.07it/s]
Epoch: 0 - loss: 0.279, acc: 0.903, val_loss: 0.272, val_acc: 0.904: 100%|██████████| 120000/120000 [00:04<00:00, 12434.98it/s]
Epoch: 1 - loss: 0.257, acc: 0.908:  97%|█████████▋| 116736/120000 [00:03<00:00, 32389.85it/s]
Epoch: 1 - loss: 0.257, acc: 0.908, val_loss: 0.257, val_acc: 0.907: 100%|██████████| 120000/120000 [00:04<00:00, 16182.11it/s]
Epoch: 2 - loss: 0.253, acc: 0.909:  99%|█████████▉| 118784/120000 [00:04<00:00, 29630.54it/s]
Epoch: 2 - loss: 0.253, acc: 0.909, val_loss: 0.250, val_acc: 0.908: 100%|██████████| 120000/120000 [00:04<00:00, 7939.42it/s]
Epoch: 3 - loss: 0.247, acc: 0.910:  98%|█████████▊| 117760/120000 [00:04<00:00, 29118.45it/s]
Epoch: 3 - loss: 0.247, acc: 0.910, val_loss: 0.248, val_acc: 0.909: 100%|██████████| 120000/120000 [00:04<00:00, 11824.69it/s]
Epoch: 4 - loss: 0.245, acc: 0.910:  99%|█████████▊| 118272/120000 [00:03<00:00, 32086.81it/s]
Epoch: 4 - los

In [24]:
test_score = model.evaluate(X_test, y_test_onehot)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])


40000/40000 [==============================] - 1s 34us/step
Test loss: 0.2823236170470715
Test accuracy: 0.90035


In [25]:
# 验证过模型可行后，用所有数据重新训练一次模型，以最大化sample size
model.fit(
    X, 
    y_onehot,
    batch_size=512, 
    epochs=40,
    validation_data=(X_validation, y_validation_onehot),
    callbacks=[TQDMCallback()],
    verbose=0
)


Epoch: 0 - loss: 0.219, acc: 0.919:  99%|█████████▉| 198144/200000 [00:06<00:00, 32380.33it/s]
Epoch: 0 - loss: 0.219, acc: 0.919, val_loss: 0.249, val_acc: 0.911: 100%|██████████| 200000/200000 [00:06<00:00, 11741.88it/s]
Epoch: 1 - loss: 0.215, acc: 0.920: 100%|█████████▉| 199680/200000 [00:06<00:00, 30955.14it/s]
Epoch: 1 - loss: 0.215, acc: 0.920, val_loss: 0.244, val_acc: 0.909: 100%|██████████| 200000/200000 [00:06<00:00, 2659.75it/s]
Epoch: 2 - loss: 0.214, acc: 0.921: 100%|█████████▉| 199168/200000 [00:06<00:00, 32539.59it/s]
Epoch: 2 - loss: 0.214, acc: 0.921, val_loss: 0.238, val_acc: 0.913: 100%|██████████| 200000/200000 [00:06<00:00, 6515.91it/s]
Epoch: 3 - loss: 0.213, acc: 0.921:  98%|█████████▊| 196608/200000 [00:06<00:00, 32527.97it/s]
Epoch: 3 - loss: 0.213, acc: 0.921, val_loss: 0.237, val_acc: 0.912: 100%|██████████| 200000/200000 [00:06<00:00, 16460.59it/s]
Epoch: 4 - loss: 0.212, acc: 0.921: 100%|█████████▉| 199680/200000 [00:06<00:00, 32160.29it/s]
Epoch: 4 - loss

In [26]:
df_submission_data = pd.read_csv('raw/test.csv.zip')
df_submission_data.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [0]:
# 把测试数据转换为numpy array 并score
X_submission = df_submission_data.iloc[:,1:].values
y_submission = model.predict_classes(X_submission)


In [29]:
print(X_submission.shape)
print(y_submission.shape)


(200000, 200)
(200000,)


In [30]:
df_submission = pd.DataFrame({'ID_code': df_submission_data['ID_code'], 'target': y_submission})
df_submission.to_csv('submission.csv', index=False)
df_submission.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,0


In [31]:
# 命令行提交
!kaggle competitions submit -c santander-customer-transaction-prediction -f submission.csv -m 'nn submission'

100% 2.56M/2.56M [00:00<00:00, 9.88MB/s]
Successfully submitted to Santander Customer Transaction Prediction